In [1]:
import pandas as pd
import numpy as np
import glob
import xarray as xr

In [2]:
main_dir = '/storage2/tkyeimiah/depot.cmc.ec.gc.ca/'
empty_list=[]
for yyyy in range(1990, 2021):
    pattern = f"{main_dir}CIS_EA_{yyyy}*.nc"
    file_paths = glob.glob(pattern)
    empty_list+=file_paths

In [3]:
empty_list2=[]
for i,v in enumerate(empty_list):
    #print(i,v)
    data = v[46:54]
    first_split = data[:4]
    second_split = data[4:6]
    third_split = data[6:]
    all_split = first_split+','+second_split+','+third_split
    empty_list2.append(all_split)

In [19]:
df= pd.DataFrame(empty_list2, columns=['date'])

# Convert the date column to datetime objects
df['date'] = pd.to_datetime(df['date'], format='%Y,%m,%d')

# Optional: split into separate columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

    
#df_new = df.drop(columns=['date'])
df.head(20)
#pd.DataFrame(np.insert(df.values, 2, values=[2, 3, 4,5], axis=0), columns=df.columns)


,date,year,month,day
0,1990-01-29,1990,1,29
1,1990-02-26,1990,2,26
2,1990-04-02,1990,4,2
3,1990-04-30,1990,4,30
4,1990-05-21,1990,5,21
5,1990-06-11,1990,6,11
6,1990-06-18,1990,6,18
7,1990-06-25,1990,6,25
8,1990-07-02,1990,7,2
9,1990-07-09,1990,7,9


In [5]:
# Save to CSV
df.to_csv('EA_dates.csv', index=False)

In [7]:
data = pd.read_csv('EA_dates.csv')

# Group by year and month, then aggregate days into a list
grouped = data.groupby(['year', 'month'])['day'].apply(list).reset_index()

# Expand the list of days into separate columns
max_days = grouped['day'].map(len).max()  # Find the maximum number of days in any month
day_columns = ['day' + str(i) for i in range(1, max_days + 1)]  # Create column names for days

# Split the days into separate columns
for i in range(max_days):
    grouped[day_columns[i]] = grouped['day'].map(lambda x: x[i] if i < len(x) else None)

# Drop the original 'day' list column
grouped.drop(columns=['day'], inplace=True)

# Display the transformed data
grouped.head(20)  # Displaying first 20 rows for brevity

# Fill NaN values with 0 and convert the day columns to integers
for col in day_columns:
    grouped[col] = grouped[col].fillna(0).astype(int)

    # Replace 0 back with NaN for presentation purposes
    grouped[col] = grouped[col].replace(0, '')

# Display the transformed data
grouped.head(10)  # Displaying first 20 rows for brevity


,year,month,day1,day2,day3,day4,day5
0,1990,1,22,15.0,29.0,1.0,8.0
1,1990,2,5,12.0,19.0,26.0,NaN
2,1990,3,5,12.0,19.0,26.0,NaN
3,1990,4,2,9.0,16.0,23.0,30.0
4,1990,5,7,14.0,21.0,28.0,NaN
5,1990,6,4,11.0,18.0,25.0,NaN
6,1990,7,2,NaN,NaN,NaN,NaN
7,1990,11,12,19.0,26.0,NaN,NaN
8,1990,12,4,11.0,18.0,25.0,NaN
9,1991,1,1,8.0,15.0,22.0,29.0


In [9]:
# Save to CSV
grouped.to_csv('EA.csv', index=False)

### creating a function for everything

In [17]:
def extract_to_csv(region):
    main_dir = '/storage2/tkyeimiah/depot.cmc.ec.gc.ca/'
    empty_list=[]
    empty_list2=[]
    for yyyy in range(1990, 2021):
        pattern = f"{main_dir}CIS_{region}_{yyyy}*.nc"
        file_paths = glob.glob(pattern)
        empty_list+=file_paths

    for i,v in enumerate(empty_list):
        data = v[46:54]
        first_split = data[:4]
        second_split = data[4:6]
        third_split = data[6:]
        all_split = first_split+','+second_split+','+third_split
        empty_list2.append(all_split)
    df= pd.DataFrame(empty_list2, columns=['date'])

    # Convert the date column to datetime objects
    df['date'] = pd.to_datetime(df['date'], format='%Y,%m,%d')

    # split into separate columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    # Group by year and month, then aggregate days into a list
    grouped = df.groupby(['year', 'month'])['day'].apply(list).reset_index()

    # Expand the list of days into separate columns
    max_days = grouped['day'].map(len).max()  # Find the maximum number of days in any month
    day_columns = ['day' + str(i) for i in range(1, max_days + 1)]  # Create column names for days

    # Split the days into separate columns
    for i in range(max_days):
        grouped[day_columns[i]] = grouped['day'].map(lambda x: x[i] if i < len(x) else None)

    # Drop the original 'day' list column
    grouped.drop(columns=['day'], inplace=True)

    # Fill NaN values with 0 and convert the day columns to integers
    for col in day_columns:
        grouped[col] = grouped[col].fillna(0).astype(int)

        # Replace 0 back with NaN for presentation purposes
        grouped[col] = grouped[col].replace(0, '')
        
    return  grouped.to_csv(f'{region}.csv', index=False), grouped.head(10)
    

In [18]:
EA_dates = extract_to_csv('EA')
EC_dates = extract_to_csv('EC')
HB_dates = extract_to_csv('HB')

In [21]:
HB_dates

(None,
    year  month  day1 day2 day3 day4 day5
 0  1990      1    29                    
 1  1990      2    26                    
 2  1990      4     2   30               
 3  1990      5    21                    
 4  1990      6    11   18   25          
 5  1990      7     2    9   16   23   30
 6  1990      8     6   13   20   27     
 7  1990      9     3   10   17   24     
 8  1990     10     1    8   15   22   29
 9  1990     11     5   12   19   26     )